[DIY Covid-19 Dashboard Kit] By Mani

# DIY Covid-19 Dashboard

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("ageVacDeath.json", "rt") as INFILE:
    data=json.load(INFILE)
jsondata=data['data']

In [4]:


def wrangle_data_young(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    final_list_young = []

    for dictionary in rawdata:
        final_death_tmp = []
        final_vac_tmp = []
        for keys,values in dictionary.items():

            if keys == 'date':
                date = (keys,dictionary[keys])


            elif keys == 'deathsAgeDemographic':
                for death_dict in dictionary[keys]:
                    death_tmp = []
                    for death_key,death_value in death_dict.items():
                        if death_key == 'age' and death_value == '00_59':
                            death_tmp.append(death_dict['deaths'])
                            final_death_tmp.append(sum(death_tmp))



            elif keys == 'vaccineAgeDemographics':
                for vac_dict in dictionary[keys]:
                    vac_tmp = []
                    for vac_key,vac_value in vac_dict.items():
                        if vac_key == 'age' and ([float(each_age.replace('+','')) for each_age in vac_value.split('_')][-1] < 60):
                            vac_tmp.append(vac_dict['newPeopleVaccinatedCompleteByVaccinationDate'])
                    final_vac_tmp.append(sum(vac_tmp))
                vac_tmp = []
                final_list_young.append([date,('DailyDeath',sum(final_death_tmp)),('DailyVac',sum(final_vac_tmp))])


    return final_list_young
    



def wrangle_data_old(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    final_list_old = []

    for dictionary in rawdata:
        final_death_tmp = []
        final_vac_tmp = []
        for keys,values in dictionary.items():

            if keys == 'date':
                date = (keys,dictionary[keys])


            elif keys == 'deathsAgeDemographic':
                for death_dict in dictionary[keys]:
                    death_tmp = []
                    for death_key,death_value in death_dict.items():
                        if death_key == 'age' and death_value == '60+':
                            death_tmp.append(death_dict['deaths'])
                            final_death_tmp.append(sum(death_tmp))



            elif keys == 'vaccineAgeDemographics':
                for vac_dict in dictionary[keys]:
                    vac_tmp = []
                    for vac_key,vac_value in vac_dict.items():
                        if vac_key == 'age' and ([float(each_age.replace('+','')) for each_age in vac_value.split('_')][0] >= 60):
                            vac_tmp.append(vac_dict['newPeopleVaccinatedCompleteByVaccinationDate'])
                    final_vac_tmp.append(sum(vac_tmp))
                vac_tmp = []
                final_list_old.append([date,('DailyDeath',sum(final_death_tmp)),('DailyVac',sum(final_vac_tmp))])
    return final_list_old
    
    


In [5]:
def getyoung_df(rawdata):
    final_data_young = [dict(item) for item in wrangle_data_young(rawdata)]

    dates=[dictionary['date'] for dictionary in rawdata]
    dates.sort()
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
#     print (startdate, ' to ', enddate)  


    index=pd.date_range(startdate, enddate, freq='D')
    young_timeseriesdf=pd.DataFrame(index=index, columns=['DailyDeath', 'DailyVac'])
    for entry in final_data_young: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['DailyDeath', 'DailyVac']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(young_timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                young_timeseriesdf.loc[date, column]=value

    # fill in any remaining "holes" due to missing dates
    young_timeseriesdf.fillna(0.0, inplace=True)
    return young_timeseriesdf





def getold_df(rawdata):
    final_data_old = [dict(item) for item in wrangle_data_old(rawdata)]

    dates=[dictionary['date'] for dictionary in rawdata]
    dates.sort()
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
#     print (startdate, ' to ', enddate)  


    index=pd.date_range(startdate, enddate, freq='D')
    old_timeseriesdf=pd.DataFrame(index=index, columns=['DailyDeath', 'DailyVac'])
    for entry in final_data_old: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['DailyDeath', 'DailyVac']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(old_timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                old_timeseriesdf.loc[date, column]=value

    # fill in any remaining "holes" due to missing dates
    old_timeseriesdf.fillna(0.0, inplace=True)
    return old_timeseriesdf






In [6]:
yg_pd_df = getyoung_df(jsondata)
old_pd_df = getold_df(jsondata)

In [7]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    from uk_covid19 import Cov19API
    import json
    
    filters = [
        'areaType=nation'
    ]

    structure = {
        "date": "date",
        "deathsAgeDemographic": "newDeaths28DaysByDeathDateAgeDemographics",
        "vaccineAgeDemographics": "vaccinationsAgeDemographics",

    }
    api = Cov19API(filters=filters, structure=structure)
    ageVacDeath=api.get_json()
    
    return ageVacDeath['data']  # return data read from the API

# Download latest data. 
##### By default loads data up to 25/10/2021

In [19]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    import time
    apibutton.icon="fa-spinner"
    apibutton.disabled=True
    apibutton.description='Downloading...'
    time.sleep(5)
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
#     apidata=access_api()
    apibutton.icon="check"
    apibutton.description='Download finished'
#         #wrangle the data and overwrite the dataframe for plotting
#     global yg_pd_df
#     global old_pd_df
#     yg_pd_df = getyoung_df(apidata)
#     old_pd_df = getold_df(apidata)
# #     refresh_graph()
    apibutton.icon,apibutton.description=("fa-hand-paper-o",'Wait10s')

    for x in reversed(range(0,10,)):
        time.sleep(1)
        apibutton.description=f'Wait {x}s'
        
        

    apibutton.disabled=False
    apibutton.icon="download"
    apibutton.description='Download data'
    
    

apibutton=wdg.Button(
    description='Get Data', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download',
)
# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='Get Data', icon='download', style=ButtonStyle(), tooltip='Click to…

# Graphs and Analysis

## Data for vaccine and deaths based on age distribution (ages <60)

In [10]:
import datetime
series=wdg.SelectMultiple(
    options=['DailyDeath','DailyVac'],
    value=['DailyDeath','DailyVac'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

dates = [datetime.date(2020,i,1) for i in range(1,13)]
options = [(i.strftime('%b'), i) for i in dates]
# print(options)
Months=wdg.SelectionRangeSlider(
    options=options,
    index=(0,11),
    description='Months(Faulty)',
    disabled=False
)
but = wdg.Button(description = 'Select Everything', button_style='info',  tooltip='To Display Vaccinations and Deaths',
)

def select_all(*args):
    series.value = series.options

but.on_click(select_all)
# try replacing HBox with a VBox
controls=wdg.HBox([series, but,scale,Months])

def timeseries_graph_young(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        yg_pd_df[list(gcols)].plot(logy=logscale)
        plt.title('Ages <60')
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   


graph=wdg.interactive_output(timeseries_graph_young, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

## Data for vaccine and deaths based on age distribution (ages 60 and above)

In [11]:
import datetime
old_series=wdg.SelectMultiple(
    options=['DailyDeath','DailyVac'],
    value=['DailyDeath','DailyVac'],
    rows=2,
    description='Stats:',
    disabled=False
)

old_scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)
however = wdg.Button(description = 'Select Everything', button_style='info',  tooltip='To Display Vaccinations and Deaths',
)

def select_all(*args):
    old_series.value = old_series.options

however.on_click(select_all)
# try replacing HBox with a VBox


old_dates = [datetime.date(2020,i,1) for i in range(1,13)]
old_options = [(i.strftime('%b'), i) for i in old_dates]
# print(options)
old_Months=wdg.SelectionRangeSlider(
    options=old_options,
    index=(0,11),
    description='Months(Faulty)',
    disabled=False
)

# try replacing HBox with a VBox
old_controls=wdg.HBox([old_series,however,old_scale,old_Months])

def timeseries_graph_old(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        old_pd_df[list(gcols)].plot(logy=logscale)
        plt.title('Ages 60 or Above')
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
old_graph=wdg.interactive_output(timeseries_graph_old, {'gcols': old_series, 'gscale': old_scale})

display(old_controls, old_graph)
# def refresh_graph():
#     """ We change the value of the widget in order to force a redraw of the graph;
#     this is useful when the data have been updated. This is a bit of a gimmick; it
#     needs to be customised for one of your widgets. """
#     current=whichwalk.value
#     if current==whichwalk.options[0]:
#         other=whichwalk.options[1]
#     else:
#         other=whichwalk.options[0]
#     whichwalk.value=other # forces the redraw
#     whichwalk.value=current # now we can change it back
    
    
# graph=wdg.interactive_output(plot_young, {'walk': whichwalk})

Output()

**Author and Copyright Notice** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*